### sparse Autoencoder for states

In [1]:
import tensorflow as tf
import keras
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import math
import copy
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
state_features = ['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']
print (state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']


In [3]:
train_df = pd.read_csv('../../data/train_scaled.csv')

In [4]:
train_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward,vaso_input,iv_input
0,0.000000,12.0,1.0,0.203388,0.000000,0.0,0.304348,0.25,0.183842,1.000000,...,0.000000,0.177479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222560,12.0,1.0,0.203388,0.000000,0.0,0.130435,0.50,0.183842,1.000000,...,0.782139,0.175772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.356608,12.0,1.0,0.203388,0.000000,0.0,0.086957,0.50,0.183842,1.000000,...,0.767260,0.174328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.452837,12.0,1.0,0.203388,0.000000,0.0,0.217391,0.50,0.183842,1.000000,...,0.000000,0.174328,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
4,0.000000,14.0,0.0,0.909010,0.142857,0.0,0.217391,0.50,0.204211,0.047619,...,0.657302,0.179999,1.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0


In [5]:
keys = train_df.keys()

In [6]:
test_df = pd.read_csv('../../data/test_scaled.csv')

In [7]:
test_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward,vaso_input,iv_input
0,0.000000,61.0,0.0,0.3779,0.230769,0.0,0.545455,0.75,0.926956,0.40625,...,0.000000,0.601591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,0.222560,61.0,0.0,0.3779,0.230769,0.0,0.363636,0.75,0.926956,0.37500,...,0.747928,0.607028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.356608,61.0,0.0,0.3779,0.230769,0.0,0.363636,1.00,0.926956,0.34375,...,0.713579,0.611279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.452837,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.31250,...,0.711891,0.615618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,0.527957,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.28125,...,0.726051,0.619879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [8]:
X_train, X_val, _, _ = train_test_split(train_df, train_df, test_size=0.2, random_state=42)

In [9]:
training_df = pd.DataFrame(X_train, columns=keys)
validating_df = pd.DataFrame(X_val, columns=keys)

In [10]:
training_df.sample(2).loc[:,state_features].head()

,Albumin,Arterial_BE,Arterial_lactate,Arterial_pH,BUN,CO2_mEqL,Calcium,Chloride,Creatinine,DiaBP,...,Weight_kg,age,elixhauser,gender,mechvent,output_4hourly,output_total,paCO2,paO2,re_admission
30477,0.421099,0.459854,0.039929,0.635088,0.712926,0.290816,0.465969,0.396579,0.385824,0.441882,...,0.308797,0.826305,0.500000,0.0,0.0,0.729513,0.778110,0.183564,0.447642,0.0
125960,0.410757,0.474453,0.048734,0.705263,0.630160,0.255102,0.418848,0.463158,0.270786,0.598824,...,0.283662,0.446361,0.285714,1.0,1.0,0.820955,0.659016,0.151781,0.241727,1.0


In [93]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.zeros([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.zeros([n_input])),
}

In [94]:
class AutoEncoder:
    def __init__(self):
        self.input_size = n_input
        self.batch_size = tf.placeholder(tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size, n_hidden_1], 0.05)
        self.X = tf.placeholder("float", [None, self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist, tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                   tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        #self.kl = tf.reduce_mean(tf.contrib.keras.losses.kullback_leibler_divergence(self.sparsity_dist, self.layer_1_a))  
    # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001 * self.kl
        #tf.summary.scalar('loss', self.loss)
        #self.accuracy = tf.reduce_mean(tf.metrics.accuracy(self.layer_1_d, self.target_y))
        #tf.summary.scalar('acc', self.acc)
        #self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [95]:
def eval_on_val():
    x = validating_df.loc[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:x, autoencoder.batch_size:X_val.shape[0]})
    print ("validation loss is: ", l)

In [105]:
batch_size = 100
autoencoder = AutoEncoder()
#tf.summary.scalar('loss', autoencoder.loss)
# tf.summary.scalar('acc', autoencoder.acc)
#merged = tf.summary.merge_all()
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    #writer = tf.summary.FileWriter('logs/sparse_auto', sess.graph)
    sess.run(init)
    # Training cycle
    for step in range(100000):
        #sum_loss = 0
        a = training_df.sample(n=batch_size)
        cur_state = a.loc[:, state_features]
        _, l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        #sum_loss += l
        #writer.add_summary(summary, step)
        if step % 1000==0 and step > 0:
            print ("step is ", step )
            #print ("total loss is ", sum_loss)
            print ("total loss is ", l)
            sum_loss = 0
        if step % 20000 == 0 and step >0:
            b = training_df.sample(n=1)
            cur_state_trial = b.loc[:, state_features]
            target_y, reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print (reconstruction)
            print (target_y)
        if step % 20000 == 0 and step >0:
            eval_on_val()
    #save_path = saver.save(sess, "sparse_autoencoder_model/model.ckpt")
    #print ('model saved to ', save_path)
    train_states = train_df.loc[:,state_features]
    train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})

    train_autoencode = pd.DataFrame(train_repr[0])
    train_autoencode['vaso_input'] = train_df['vaso_input']
    train_autoencode['iv_input'] = train_df['iv_input']
    train_autoencode['reward'] = train_df['reward']
    train_autoencode['icustayid'] = train_df['icustayid']

    test_states = test_df.loc[:,state_features]
    test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
    test_autoencode = pd.DataFrame(test_repr[0])
    test_autoencode['vaso_input'] = test_df['vaso_input']
    test_autoencode['iv_input'] = test_df['iv_input']
    test_autoencode['reward'] = test_df['reward']
    test_autoencode['icustayid'] = test_df['icustayid']

step is  1000
total loss is  0.0982328
step is  2000
total loss is  0.0733788
step is  3000
total loss is  0.0571428
step is  4000
total loss is  0.050553
step is  5000
total loss is  0.0441328
step is  6000
total loss is  0.0260805
step is  7000
total loss is  0.0235437
step is  8000
total loss is  0.0196456
step is  9000
total loss is  0.0180331
step is  10000
total loss is  0.0156168
step is  11000
total loss is  0.0125812
step is  12000
total loss is  0.0113878
step is  13000
total loss is  0.00986484
step is  14000
total loss is  0.00885993
step is  15000
total loss is  0.00770599
step is  16000
total loss is  0.00676957
step is  17000
total loss is  0.00612195
step is  18000
total loss is  0.00518439
step is  19000
total loss is  0.00487075
step is  20000
total loss is  0.00416981
[[ 0.27022541  0.41355208  0.02706414  0.74386322  0.46368784  0.21767643
   0.40284488  0.44468108  0.26737663  0.54049104  0.05323806  0.97447157
   0.18100199  0.13828522  0.35282469  0.64310336  0.4

step is  86000
total loss is  0.000453062
step is  87000
total loss is  0.000451573
step is  88000
total loss is  0.000419774
step is  89000
total loss is  0.000491614
step is  90000
total loss is  0.000395685
step is  91000
total loss is  0.000403904
step is  92000
total loss is  0.000451452
step is  93000
total loss is  0.000380449
step is  94000
total loss is  0.000404947
step is  95000
total loss is  0.000381785
step is  96000
total loss is  0.000418023
step is  97000
total loss is  0.000402506
step is  98000
total loss is  0.000427586
step is  99000
total loss is  0.000392169


In [107]:
train_autoencode.to_csv("../../data/train_scaled_encoded.csv", index=False)
test_autoencode.to_csv("../../data/test_scaled_encoded.csv", index=False)

### Swap vaso and iv input, fixing glitch

In [12]:
train_encoded = pd.read_csv('../../data/train_scaled_encoded.csv')
test_encoded = pd.read_csv('../../data/test_scaled_encoded.csv')

In [14]:
temp = train_encoded['vaso_input']
train_encoded['vaso_input'] = train_encoded['iv_input']
train_encoded['iv_input'] = temp

In [15]:
train_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,vaso_input,iv_input,reward,icustayid
0,0.029027,0.061212,0.224086,0.079448,0.052524,0.077151,0.103817,0.063714,0.032102,0.042459,...,0.083823,0.053838,0.081020,0.057297,0.030372,0.047363,0,0.0,0.0,12.0
1,0.031518,0.078677,0.120369,0.040228,0.047623,0.104624,0.093498,0.073046,0.035958,0.056553,...,0.088714,0.073047,0.083202,0.043883,0.036930,0.057726,0,0.0,0.0,12.0
2,0.030942,0.083416,0.106333,0.040481,0.039120,0.098819,0.077658,0.074696,0.041053,0.053553,...,0.092754,0.079161,0.082560,0.041910,0.036587,0.067986,0,0.0,0.0,12.0
3,0.042970,0.050963,0.090261,0.093201,0.031332,0.057674,0.032040,0.054802,0.061447,0.053960,...,0.098049,0.074196,0.088113,0.045784,0.041096,0.129189,0,0.0,15.0,12.0
4,0.064572,0.084793,0.065976,0.070753,0.063990,0.051591,0.050112,0.049624,0.092619,0.080058,...,0.032011,0.081824,0.037519,0.061251,0.071471,0.039540,0,4.0,0.0,14.0


In [16]:
temp = test_encoded['vaso_input']
test_encoded['vaso_input'] = test_encoded['iv_input']
test_encoded['iv_input'] = temp

In [17]:
test_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,vaso_input,iv_input,reward,icustayid
0,0.044787,0.024944,0.059501,0.186372,0.099174,0.010116,0.015910,0.031792,0.063170,0.050696,...,0.037952,0.041529,0.046699,0.054380,0.082892,0.012378,0,4.0,0.0,61.0
1,0.048257,0.029508,0.033046,0.102542,0.107019,0.015018,0.014971,0.034197,0.057205,0.072662,...,0.037210,0.051388,0.044145,0.039312,0.096143,0.013473,0,4.0,0.0,61.0
2,0.041906,0.028968,0.028705,0.111719,0.098205,0.013579,0.012648,0.027884,0.074506,0.081935,...,0.037256,0.055131,0.042269,0.039004,0.103266,0.015244,0,4.0,0.0,61.0
3,0.040383,0.029019,0.029167,0.131740,0.099840,0.013389,0.012455,0.028260,0.072425,0.088626,...,0.037022,0.057600,0.039294,0.038362,0.117384,0.015660,0,4.0,0.0,61.0
4,0.045685,0.029040,0.027817,0.137894,0.106901,0.013968,0.012697,0.028393,0.073113,0.088150,...,0.036816,0.058196,0.038383,0.038041,0.123431,0.014925,0,4.0,0.0,61.0


In [18]:
train_encoded.to_csv("../../data/train_scaled_encoded.csv", index=False)
test_encoded.to_csv("../../data/test_scaled_encoded.csv", index=False)